In [ ]:
import json
from typing import Dict, Set

import pandas as pd
import yaml
from IPython.display import display
from rapidfuzz import fuzz

pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", 50)

psg_directory = "../data/geography/"
psg_data_file = "psgc_2025-08-07.csv"

In [2]:
df = pd.read_csv(psg_directory + psg_data_file)
display(df.info())
display(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43769 entries, 0 to 43768
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   psgc_id                43769 non-null  int64  
 1   name                   43769 non-null  object 
 2   correspondence_code    43719 non-null  float64
 3   geographic_level       43767 non-null  object 
 4   old_names              1699 non-null   object 
 5   city_class             149 non-null    object 
 6   income_classification  1724 non-null   object 
 7   settlement_type        42011 non-null  object 
 8   population             43762 non-null  object 
 9   Unnamed: 9             75 non-null     object 
 10  barangay_status        2855 non-null   object 
dtypes: float64(1), int64(1), object(9)
memory usage: 3.7+ MB


None

,psgc_id,name,correspondence_code,geographic_level,old_names,city_class,income_classification,settlement_type,population,Unnamed: 9,barangay_status
0,1300000000,National Capital Region (NCR),130000000.0,Reg,NaN,NaN,NaN,NaN,"13,484,462",NaN,NaN
1,1380100000,City of Caloocan,137501000.0,City,NaN,HUC,1st,NaN,"1,661,584",NaN,NaN
2,1380100001,Barangay 1,137501001.0,Bgy,NaN,NaN,NaN,U,"2,319",NaN,NaN
3,1380100002,Barangay 2,137501002.0,Bgy,NaN,NaN,NaN,U,"5,156",NaN,NaN
4,1380100003,Barangay 3,137501003.0,Bgy,NaN,NaN,NaN,U,"2,497",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
43764,1999908006,Manaulanan,124712037.0,Bgy,NaN,NaN,NaN,U,"7,632",NaN,NaN
43765,1999908007,Pamalian,124712062.0,Bgy,NaN,NaN,NaN,R,"3,256",NaN,NaN
43766,1999908008,Tapodoc,124717017.0,Bgy,NaN,NaN,NaN,R,"1,767",NaN,NaN
43767,1999908009,Macabual,124712034.0,Bgy,NaN,NaN,NaN,R,"4,557",NaN,NaN


In [ ]:
# this code is just copied from my barangay project so there are more explanations there
# i think

df["psgc_id"] = df["psgc_id"].astype(str).str.zfill(10)
df = df.map(lambda x: x.strip() if isinstance(x, str) else x)

geographic_level_map = {
    "Reg": "region",
    "City": "city",
    "Mun": "municipality",
    "Prov": "province",
    "SubMun": "submunicipality",
    "Bgy": "barangay",
}
df["geographic_level"] = df["geographic_level"].replace(geographic_level_map)

df["barangay_code"] = df["psgc_id"].str[-3:]
df["municipal_or_city_code"] = df["psgc_id"].str[-5:-3]
df["province_or_huc_code"] = df["psgc_id"].str[-8:-5]
df["region_code"] = df["psgc_id"].str[-10:-8]

df["barangay_mapper"] = df["psgc_id"].str[-10:]
df["municipal_or_city_mapper"] = df["psgc_id"].str[-10:-3]
df["province_or_huc_mapper"] = df["psgc_id"].str[-10:-5]
df["region_mapper"] = df["psgc_id"].str[-10:-8]

df.sample(10)

regions_filter = (
    (df["province_or_huc_code"] == "000")
    & (df["municipal_or_city_code"] == "00")
    & (df["barangay_code"] == "000")
)
regions_mapper = (
    df.loc[regions_filter, ["region_mapper", "name"]]
    .sort_values("region_mapper")
    .set_index("region_mapper", drop=True)
    .to_dict()["name"]
)


province_or_huc_filter = (
    ~(df["province_or_huc_code"] == "000")
    & (df["municipal_or_city_code"] == "00")
    & (df["barangay_code"] == "000")
)

province_or_huc_mapper = (
    df.loc[province_or_huc_filter, ["province_or_huc_mapper", "name"]]
    .sort_values("province_or_huc_mapper")
    .set_index("province_or_huc_mapper")
    .to_dict()["name"]
)
municipal_or_city_filter = (
    ~(df["province_or_huc_code"] == "000")
    & ~(df["municipal_or_city_code"] == "00")
    & (df["barangay_code"] == "000")
)

municipal_or_city_mapper = (
    df.loc[municipal_or_city_filter, ["municipal_or_city_mapper", "name"]]
    .sort_values("municipal_or_city_mapper")
    .set_index("municipal_or_city_mapper")
    .to_dict()["name"]
)

df["region"] = df["region_mapper"].map(regions_mapper)
df["province_or_huc"] = df["province_or_huc_mapper"].map(province_or_huc_mapper)
df["municipality_or_city"] = df["municipal_or_city_mapper"].map(
    municipal_or_city_mapper
)
display(df.sample(10))

,psgc_id,name,correspondence_code,geographic_level,old_names,city_class,income_classification,settlement_type,population,Unnamed: 9,barangay_status,barangay_code,municipal_or_city_code,province_or_huc_code,region_code,barangay_mapper,municipal_or_city_mapper,province_or_huc_mapper,region_mapper,region,province_or_huc,municipality_or_city
5840,0105530018,Maambal,15530018.0,barangay,NaN,NaN,NaN,R,"1,429",NaN,NaN,018,30,055,01,0105530018,0105530,01055,01,Region I (Ilocos Region),Pangasinan,Pozorrubio
7407,0203103018,Malasin,23103018.0,barangay,NaN,NaN,NaN,R,"1,778",NaN,NaN,018,03,031,02,0203103018,0203103,02031,02,Region II (Cagayan Valley),Isabela,Aurora
21097,0506210024,Sipaya,56210024.0,barangay,NaN,NaN,NaN,R,757,NaN,NaN,024,10,062,05,0506210024,0506210,05062,05,Region V (Bicol Region),Sorsogon,Juban
13019,0401029020,Niogan,41029020.0,barangay,NaN,NaN,NaN,R,401,NaN,NaN,020,29,010,04,0401029020,0401029,04010,04,Region IV-A (CALABARZON),Batangas,Taal
22247,0600617006,Importante,60617006.0,barangay,NaN,NaN,NaN,R,"1,540",NaN,NaN,006,17,006,06,0600617006,0600617,06006,06,Region VI (Western Visayas),Antique,Tibiao
38176,1108210004,Cogonon,118210004.0,barangay,NaN,NaN,NaN,R,"1,233",NaN,NaN,004,10,082,11,1108210004,1108210,11082,11,Region XI (Davao Region),Davao de Oro,New Bataan
22375,0601904029,Poblacion Ilaya,61904029.0,barangay,NaN,NaN,NaN,U,"1,959",NaN,NaN,029,04,019,06,0601904029,0601904,06019,06,Region VI (Western Visayas),Capiz,Dumarao
23261,0603016014,Gutao,63016014.0,barangay,NaN,NaN,NaN,R,823,NaN,NaN,014,16,030,06,0603016014,0603016,06030,06,Region VI (Western Visayas),Iloilo,Dingle
15313,0405622095,Rizal (Poblacion),45622095.0,barangay,Poblacion,NaN,NaN,U,"3,432",NaN,NaN,095,22,056,04,0405622095,0405622,04056,04,Region IV-A (CALABARZON),Quezon,Lopez
23293,0603017011,Buenavista,63017011.0,barangay,NaN,NaN,NaN,R,580,NaN,NaN,011,17,030,06,0603017011,0603017,06030,06,Region VI (Western Visayas),Iloilo,Dueñas


In [ ]:
clean_mun = (
    df["municipality_or_city"]
    .astype(str)
    .str.lower()
    .str.replace(" ", "")
    .str.replace("(POB.)", "")
)
clean_name = (
    df["name"].astype(str).str.lower().str.replace(" ", "").str.replace("(POB.)", "")
)

df["candidate_hook"] = clean_mun + clean_name

In [5]:
df.sample(10)

,psgc_id,name,correspondence_code,geographic_level,old_names,city_class,income_classification,settlement_type,population,Unnamed: 9,barangay_status,barangay_code,municipal_or_city_code,province_or_huc_code,region_code,barangay_mapper,municipal_or_city_mapper,province_or_huc_mapper,region_mapper,region,province_or_huc,municipality_or_city,candidate_hook
20924,0506206008,Canjela,56206008.0,barangay,NaN,NaN,NaN,R,881,NaN,NaN,008,06,062,05,0506206008,0506206,05062,05,Region V (Bicol Region),Sorsogon,Castilla,castillacanjela
26707,0701222006,Cambuyo,71222006.0,barangay,NaN,NaN,NaN,R,650,NaN,NaN,006,22,012,07,0701222006,0701222,07012,07,Region VII (Central Visayas),Bohol,Garcia Hernandez,garciahernandezcambuyo
19467,0501723029,San Luis,51723029.0,barangay,NaN,NaN,NaN,U,359,NaN,Pob.,029,23,017,05,0501723029,0501723,05017,05,Region V (Bicol Region),Camarines Sur,Nabua,nabuasanluis
23511,0603023029,Mangil,63023029.0,barangay,NaN,NaN,NaN,R,"1,244",NaN,NaN,029,23,030,06,0603023029,0603023,06030,06,Region VI (Western Visayas),Iloilo,Janiuay,janiuaymangil
14987,0405610038,San Vicente Silangan,45610038.0,barangay,NaN,NaN,NaN,R,828,NaN,NaN,038,10,056,04,0405610038,0405610,04056,04,Region IV-A (CALABARZON),Quezon,Catanauan,catanauansanvicentesilangan
2090,1401103001,Ampusongan,141103001.0,barangay,NaN,NaN,NaN,R,"2,532",NaN,NaN,001,03,011,14,1401103001,1401103,14011,14,Cordillera Administrative Region (CAR),Benguet,Bakun,bakunampusongan
41930,1903621006,Bobo,153621006.0,barangay,NaN,NaN,NaN,R,"1,013",NaN,NaN,006,21,036,19,1903621006,1903621,19036,19,Bangsamoro Autonomous Region In Muslim Mindana...,Lanao del Sur,Piagapo,piagapobobo
17890,0500503041,Pandan,50503041.0,barangay,NaN,NaN,NaN,R,"1,930",NaN,NaN,041,03,005,05,0500503041,0500503,05005,05,Region V (Bicol Region),Albay,Daraga,daragapandan
9909,0304907004,Bitulok,34907004.0,barangay,North Pob.,NaN,NaN,R,"2,918",NaN,NaN,004,07,049,03,0304907004,0304907,03049,03,Region III (Central Luzon),Nueva Ecija,Gabaldon,gabaldonbitulok
7409,0203103020,Villa Nuesa,23103020.0,barangay,NaN,NaN,NaN,R,356,NaN,NaN,020,03,031,02,0203103020,0203103,02031,02,Region II (Cagayan Valley),Isabela,Aurora,auroravillanuesa


In [ ]:
from typing import List


def sanitize_input(input_str: str, exclude: List[str] | str | None = None) -> str:
    """
    Removes whitespaces, lowers, and remove all strings listed in exclude
    """
    sanitized_str = input_str.lower()
    if exclude is None:
        return sanitized_str

    if isinstance(exclude, list):
        exclude = [x.lower() for x in exclude if isinstance(x, str)]
        for item in exclude:
            sanitized_str.replace(item, "")
        return sanitized_str

    return sanitized_str.replace(exclude.lower(), "")

In [ ]:
input_str = "BACARRALIBTONG"
sanitized_input = sanitize_input(input_str)

df["sanitized_candidate_hook"] = df["candidate_hook"].apply(
    sanitize_input, args=("(pob.)",)
)
df["score"] = (
    df["sanitized_candidate_hook"].apply(fuzz.ratio, args=(sanitized_input,)).round(1)
)

In [8]:
df["score"].value_counts().reset_index().sort_values(by="score", ascending=False)

,score,count
169,100.0,1
183,80.0,1
168,76.9,1
167,74.3,1
154,72.0,2
...,...,...
184,8.7,1
171,8.3,1
165,7.7,2
186,7.4,1


I used excel to parse the PDF... after long battle, didn't work...

Now let's use tabula-py (port of tabula from java)

In [9]:
# let's try tabula-py
import tabula

In [10]:
df = tabula.read_pdf("../data/education/masterlist.pdf", pages="all")

Failed to import jpype dependencies. Fallback to subprocess.
No module named 'jpype'


In [ ]:
from tqdm import tqdm

root_df = pd.DataFrame()
for idx, d in enumerate(tqdm(df)):
    d["page"] = idx
    root_df = pd.concat([root_df, d])


100%|██████████| 544/544 [00:02<00:00, 199.75it/s] 


In [12]:
root_df = root_df.reset_index(drop=True)

In [14]:
educdf = root_df[root_df["BEIS School ID"].str.strip().str.isnumeric().notna()]

# Data Cleaning!

In [17]:
root_df.sample(10)

,Region,Division,District,BEIS School ID,School Name,Street Address,Municipality,Legislative District,Barangay,Sector,Urban/Ru,Sacl hColaosls Sifuicbactliaosnsification,Modified Curricural Offering Classification,page
19,CAR,Apayao,Calanasan,135239,Milagrosa ES,-Milagrosa,CALANASAN (BAYAG),Lone District,EVA PUZON,Public,Partially U,bDaenpED Managed,Purely ES,501
110,Region II,Isabela,Luna,103490,Pulay-Union Kalinga Elementary School,Medina,LUNA,3rd District,PULAY,Public,Partially U,bDaenpED Managed,Purely ES,42
105,Region I,Ilocos Norte,Banna (Espirit,)300008,Caestebanan NHS,"14 Sinamar, Banna, Ilocos Norte",BANNA (ESPIRITU),2nd District,SINAMAR,Public,Partially U,bDaenpED Managed,JHS with SHS,0
110,Region VI,Antique,Tobias Fornier,115382,Opsan PS,OPSAN (SAN JOSE),TOBIAS FORNIER (DAO),Lone District,OPSAN (SAN JOSE),Public,Partially U,bDaenpED Managed,Purely ES,232
64,Region XI,Davao City,Marilog,304384,Marahan National High School,"-Sitio Marahan, Marilog Dist, D.C",DAVAO CITY,3rd District,VICENTE HIZON SR.,Public,Partially U,bDaenpED Managed,JHS with SHS,419
47,Region VI,Passi City,Passi City Distr,ct II (East)117914,Prospero Palomo Elementary School,"Brgy. Sarapan, Passi City",CITY OF PASSI,4th District,SARAPAN,Public,Partially U,bDaenpED Managed,Purely ES,267
90,Region I,"Pangasinan I, Lingayen",Sta. Barbara I,101634,Maronong ES,MARONONG,SANTA BARBARA,3rd District,MARONONG,Public,Urban,DepED Managed,Purely ES,18
33,Region IV-B,Palawan,Magsaysay,408878,"Kai Messiah Christian Academy, Inc.","Brgy. Emilod, Magsaysay, Palawan",MAGSAYSAY,1st District,EMILOD,Private,Rural,Non-Sectarian,Purely ES,174
32,Region IV-B,Oriental Mindoro,Pinamalayan E,st110558,Juan Morente Sr. Mem. Pilot Sch.,"Mabini St.,",PINAMALAYAN,2nd District,ZONE IV (POB.),Public,Partially U,bDaenpED Managed,Purely ES,169
106,Region II,Isabela,Luna,103486,Lalog Elementary School,-,LUNA,3rd District,LALOG 1,Public,Partially U,bDaenpED Managed,Purely ES,42


In [21]:
for col in root_df.columns:
    print(col)

Region
Division
District
BEIS School ID
School Name
Street Address
Municipality
Legislative District
Barangay
Sector
Urban/Ru
Sacl hColaosls Sifuicbactliaosnsification
Modified Curricural Offering Classification
page


In [22]:
correct_column_names = {
    "Region": "region",
    "Division": "division",
    "District": "district",
    "BEIS School ID": "beis_school_id",
    "School Name": "school_name",
    "Street Address": "street_address",
    "Municipality": "municipality",
    "Legislative District": "legislative_district",
    "Barangay": "barangay",
    "Sector": "sector",
    "Urban/Ru": "settlement_type",
    "Sacl hColaosls Sifuicbactliaosnsification": "school_subclassification",
    "Modified Curricural Offering Classification": "modified_cultural_offering_classification",
    "page": "page",
}

In [25]:
root_df = root_df.rename(correct_column_names, axis=1)

# Fixing Categories
There are a few categorical data in here that got messed up during the parsing of the PDF File

In [30]:
root_df.sample(10)

,region,division,district,beis_school_id,school_name,street_address,municipality,legislative_district,barangay,sector,settlement_type,school_subclassification,modified_cultural_offering_classification,page
110,Region VIII,Eastern Samar,Guiuan East,404742,"St. Mary’s Academy of Guiuan, Inc.",San Jose Street,GUIUAN,Lone District,POBLACION WARD 7,Private,Partially U,bSeacntarian,JHS with SHS,328
16,Region X,Misamis Oriental,Libertad,501321,Lubluban IS,P2 Lubluban,LIBERTAD,2nd District,LUBLUBAN,Public,Partially U,bDaenpED Managed,ES and JHS (K to 10),393
7,NCR,Quezon City,School District,X482521,Another Home to Grow Learning Center,#24 Adrian,QUEZON CITY,5th District,NORTH FAIRVIEW,Private,Urban,Non-Sectarian,Purely ES,524
72,Region I,Ilocos Sur,San Vicente,100615,Pudoc Elementary School,"Pudoc, San Vicente, Ilocos Sur",SAN VICENTE,1st District,PUDOC,Public,Partially U,bDaenpED Managed,Purely ES,6
63,Region VI,Negros Occidental,Calatrava II,116939,Winaswasan Elementary School,Winaswasan Proper,CALATRAVA,1st District,WINASWASAN,Public,Partially U,bDaenpED Managed,Purely ES,252
5,BARMM,Lanao del Sur - I,Masiu B Distri,t318125,Masiu Science and Techonology HS,"-Maruhom,Marandacan putad",MASIU,1st District,PUTAD MARANDANG BUISAN,Public,Partially U,bDaenpED Managed,Purely JHS,480
26,Region VI,Iloilo,Janiuay II,116419,Panuran Primary School,Bgry. Panuran,JANIUAY,3rd District,PANURAN,Public,Partially U,bDaenpED Managed,Purely ES,247
69,Region X,Lanao del Norte,Bacolod,405103,Bible Baptist Christian Academy of Maranding L...,Purok Nilo,LALA,2nd District,MARANDING,Private,Partially U,bSeacntarian,All Offering (K to 12),384
1,Region IV-B,Oriental Mindoro,Gloria,110455,Tambong ES,"Punta Tambong, Gloria, Oriental Mindoro",GLORIA,2nd District,TAMBONG,Public,Partially U,bDaenpED Managed,Purely ES,168
68,Region VI,Kabankalan City,Kabankalan III,445504,"Daan Banua FBC Learning Center, Inc.",Purok 1,CITY OF KABANKALAN,6th District,DAAN BANUA,Private,Partially U,bSeacntarian,Purely ES,266


In [ ]:
root_df["sector"].value_counts(dropna=False)
# looks clean, bet lets convert to snake case

sector
Public       47421
Private      13256
SUCs/LUCs      247
Name: count, dtype: int64

In [ ]:
root_df["sector"] = root_df["sector"].replace("Public", "public")
root_df["sector"] = root_df["sector"].replace("Private", "private")
root_df["sector"] = root_df["sector"].replace("SUCs/LUCs", "suc_luc")

In [48]:
root_df.sample(10)

,region,division,district,beis_school_id,school_name,street_address,municipality,legislative_district,barangay,sector,settlement_type,school_subclassification,modified_cultural_offering_classification,page
18,Region VIII,Eastern Samar,Oras West,122670,Trinidad Elementary School,Brgy Road,ORAS,Lone District,TRINIDAD (MAYCOROT),public,partially_urban,bDaenpED Managed,Purely ES,330
73,BARMM,Lanao del Sur - II,Sultan Guman,er133822,Biosong Elementary School,"Biosong, PICONG LANAO SUR II",SULTAN GUMANDER (PICONG),2nd District,BIASONG,public,partially_urban,bDaenpED Managed,Purely ES,483
91,Region IX,Zamboanga del Sur,Kumalarang,125110,Francisco A.Ongcol ES,Purok-3,KUMALARANG,2nd District,SUMINALUM,public,partially_urban,bDaenpED Managed,Purely ES,360
33,Region IV-A,Imus City,Imus I,424029,Academia Primera School,"Anabu II-C Imus City, Cavite",IMUS,3rd District,ANABU I-C,private,urban,Non-Sectarian,ES and JHS (K to 10),156
7,Region IV-A,Laguna,Los Baños,402521,Christian Bible Baptist Academy,Nature's Ville,LOS BAÑOS,2nd District,ANOS,private,urban,Sectarian,All Offering (K to 12),122
81,CARAGA,Bayugan City,Bayugan Nort,212003,New Katipunan ES,"Purok 11, San Juan, Bayugan City",CITY OF BAYUGAN,1st District,SAN JUAN,public,partially_urban,bDaenpED Managed,Purely ES,472
82,Region I,"Pangasinan I, Lingayen",Lingayen I,101487,Namolan ES,namolan,LINGAYEN (Capital),2nd District,NAMOLAN,public,urban,DepED Managed,Purely ES,16
103,Region VIII,Leyte,Jaro I,121463,Malobago Elementary School,N/A,JARO,2nd District,MALOBAGO,public,partially_urban,bDaenpED Managed,Purely ES,317
35,Region V,Albay,Daraga South,301832,Lacag National High School,"-Lacag, Daraga, Albay",DARAGA (LOCSIN),2nd District,LACAG,public,partially_urban,bDaenpED Managed,JHS with SHS,185
27,Region IV-A,Cavite,Magallanes,402229,"Magallanes Western Cavite Institute, Inc.",#001 DE GUIA ST.,MAGALLANES,7th District,BARANGAY 3 (POB.),private,partially_urban,bNaonn-Sectarian,Purely JHS,116


In [ ]:
root_df["settlement_type"].value_counts(dropna=False)
# lets correct categories

settlement_type
Partially U    47606
Urban          10404
Rural           2914
Name: count, dtype: int64

In [ ]:
root_df["settlement_type"] = root_df["settlement_type"].replace(
    "Partially U", "partially_urban"
)
root_df["settlement_type"] = root_df["settlement_type"].replace("Urban", "urban")
root_df["settlement_type"] = root_df["settlement_type"].replace("Rural", "rural")


In [36]:
root_df["settlement_type"].value_counts()

settlement_type
partially_urban    47606
urban              10404
rural               2914
Name: count, dtype: int64

In [ ]:
root_df["school_subclassification"].value_counts(dropna=False)
# now, that's dirty

school_subclassification
bDaenpED Managed                 40457
DepED Managed                     6795
Non-Sectarian                     5244
bNaonn-Sectarian                  4252
bSeacntarian                      2595
Sectarian                         1163
bSUanC Managed                     160
bLoacnal Government                109
SUC Managed                         50
Local Government                    45
bLUanC                              21
LUC                                 16
bDaOnST Managed                      9
DOST Managed                         4
bLoacnal International School        2
bOatnher GA Managed                  1
Other GA Managed                     1
Name: count, dtype: int64

Here, I have to double check the real values in the PDF via reading it manually. Then I'll create the dictionary renamer later

In [52]:
root_df[root_df["school_subclassification"] == "bSUanC Managed"].sample(2)

,region,division,district,beis_school_id,school_name,street_address,municipality,legislative_district,barangay,sector,settlement_type,school_subclassification,modified_cultural_offering_classification,page
100,Region IV-A,Quezon,Tagkawayan,600149,Southern Luzon State University-Judge Guillerm...,Brgy. Rizal,TAGKAWAYAN,4th District,RIZAL,suc_luc,partially_urban,bSUanC Managed,Purely JHS,134
12,Region IX,Zamboanga del Norte,Katipunan I,600087,Jose Rizal Memorial State University - Katipun...,"Brgy. Dos (Pob), Katipunan",KATIPUNAN,2nd District,BARANGAY DOS (POB.),suc_luc,partially_urban,bSUanC Managed,JHS with SHS,352


In [57]:
root_df[root_df["school_subclassification"] == "bLoacnal Government"].sample(2)

,region,division,district,beis_school_id,school_name,street_address,municipality,legislative_district,barangay,sector,settlement_type,school_subclassification,modified_cultural_offering_classification,page
100,Region VII,Cebu,Bantayan II,119065,Kabangbang Central ES,Centro,BANTAYAN,4th District,KABANGBANG,public,partially_urban,bLoacnal Government,Purely ES,282
31,Region VI,Capiz,Panit-An,501099,Felixberto Dicon Dorado Sr. Integrated School,"Ensenagan,Panitan, Capiz",PANITAN,1st District,ENSENAGAN,public,partially_urban,bLoacnal Government,ES and JHS (K to 10),236


In [58]:
root_df[root_df["school_subclassification"] == "bDaOnST Managed"].sample(2)

,region,division,district,beis_school_id,school_name,street_address,municipality,legislative_district,barangay,sector,settlement_type,school_subclassification,modified_cultural_offering_classification,page
5,Region IX,Dipolog City,Dipolog City E,st305575,Philippine Science High School - Zamboanga Pen...,Cogon,DIPOLOG CITY (Capital),2nd District,COGON,public,partially_urban,bDaOnST Managed,Purely JHS,371
13,Region I,Ilocos Sur,Sto. Domingo-,an Ildefons3o30511,PHI SCIENCE HIGH SCHOOL - ILOCOS REGION CAMPUS,Pob. East San Ildefonso Ilocos Sur,SAN ILDEFONSO,1st District,POBLACION EAST,public,partially_urban,bDaOnST Managed,JHS with SHS,8


In [ ]:
root_df[root_df["school_subclassification"] == "bLoacnal International School"].sample(
    2
)

,region,division,district,beis_school_id,school_name,street_address,municipality,legislative_district,barangay,sector,settlement_type,school_subclassification,modified_cultural_offering_classification,page
12,Region V,Camarines Norte,Labo East,409756,"ADR Bicol International Technological College,...",P-4,LABO,1st District,MALASUGUI,private,partially_urban,bLoacnal International School,Purely SHS,191
13,Region V,Camarines Norte,Labo East,409757,"Camarines Norte International School, Inc.","Maharlika Highway, P-1",LABO,1st District,MASALONG,private,partially_urban,bLoacnal International School,Purely SHS,191


In [61]:
root_df[root_df["school_subclassification"] == "bOatnher GA Managed"].sample()

,region,division,district,beis_school_id,school_name,street_address,municipality,legislative_district,barangay,sector,settlement_type,school_subclassification,modified_cultural_offering_classification,page
4,Region II,Tuguegarao City,Tuguegarao,est Educatio1n0a0l 9Z9o4,eDepartment of Agriculture R02 Child Developme...,"Nursery Compound, San Gabriel, Tuguegarao City",TUGUEGARAO CITY(Capital),3rd District,SAN GABRIEL,public,partially_urban,bOatnher GA Managed,Purely ES,54


In [63]:
mapper = {
    "bDaenpED Managed": "deped_managed",
    "DepED Managed": "deped_managed",
    "Non-Sectarian": "non_sectarian",
    "bNaonn-Sectarian": "non_sectarian",
    "bSeacntarian": "sectarian",
    "Sectarian": "sectarian",
    "bSUanC Managed": "suc_managed",
    "bLoacnal Government": "local_government",
    "SUC Managed": "suc_managed",
    "Local Government": "local_government",
    "bLUanC": "luc",
    "LUC": "luc",
    "bDaOnST Managed": "dost_managed",
    "DOST Managed": "dost_managed",
    "bLoacnal International School": "local_international_school",
    "bOatnher GA Managed": "other_ga_managed",
    "Other GA Managed": "other_ga_managed",
}

# nice list


In [64]:
# now let's replace

for key, value in mapper.items():
    root_df["school_subclassification"] = root_df["school_subclassification"].replace(
        key, value
    )

In [65]:
root_df.sample(10)

,region,division,district,beis_school_id,school_name,street_address,municipality,legislative_district,barangay,sector,settlement_type,school_subclassification,modified_cultural_offering_classification,page
9,Region VIII,Samar (Western Samar),Pinabacdao,123665,Dolores ES,Purok 3,PINABACDAO,2nd District,DOLORES,public,partially_urban,deped_managed,Purely ES,341
45,Region III,Zambales,Masinloc,106896,San Salvador Elementary School,"San Salvador, Masinloc",MASINLOC,2nd District,SAN SALVADOR,public,partially_urban,deped_managed,Purely ES,91
59,Region VII,Cebu,Daanbantaya,II119286,Purgason ES,"Purgason, Maya, Daanbantayan, Cebu",DAANBANTAYAN,4th District,MAYA,public,partially_urban,deped_managed,Purely ES,285
5,Region XII,General Santos City,Conel,131282,Arizala ES,"Purok 9 Paradise, Mabuhay, General Santos City",GENERAL SANTOS CITY (DADIANGAS),1st District,MABUHAY,public,partially_urban,deped_managed,Purely ES,450
102,NCR,Quezon City,School District,VII136547,Melencio M. Castelo ES,"-Ilang-Ilang, Payatas A",QUEZON CITY,2nd District,PAYATAS,public,urban,deped_managed,Purely ES,522
18,Region VIII,Northern Samar,San Isidro,303577,San Isidro Agro-Industrial High School,Maharlika Highway,SAN ISIDRO,1st District,SAN ROQUE,public,partially_urban,deped_managed,JHS with SHS,333
97,Region XI,Davao City,Tugbok,405513,Bright Angels of Tomorrow Academy (BATA) and P...,Green Meadows Subdivision,DAVAO CITY,1st District,TUGBOK (POB.),private,partially_urban,non_sectarian,Purely ES,421
56,Region XI,Davao De Oro,Pantukan,128549,Matiao ES,Canipa,PANTUKAN,2nd District,MATIAO,public,partially_urban,deped_managed,Purely ES,407
15,Region III,Balanga City,Balanga City E,st400601,Saint Joseph Child Development Center Inc.,#687 Macopa Street,CITY OF BALANGA (Capital),2nd District,DOÃ‘A FRANCISCA,private,partially_urban,sectarian,Purely ES,101
10,Region IV-A,Tanauan City,Tanauan City,ast II321604,Dr. Alcantara National High School,"Mabini, Tanauan City, Batangas",CITY OF TANAUAN,3rd District,MABINI,public,urban,deped_managed,Purely JHS,152


In [66]:
list_of_clean_dfs: List[pd.DataFrame] = []

In [ ]:
null_beis_school_id = root_df["beis_school_id"].isna()
isnumeric_filter = root_df["beis_school_id"].astype


In [95]:
null_beis_school_id.value_counts(dropna=False)

beis_school_id
False    60924
Name: count, dtype: int64

In [97]:
isnumeric_filter.value_counts(dropna=False)

beis_school_id
NaN      27328
True     18427
False    15169
Name: count, dtype: int64

In [85]:
import numpy as np

In [94]:
root_df["beis_school_id"].sample(10)

63                     123186
64                     109901
107                    104802
25                     300313
79                alona117028
13     Central Dis3tr2ic0t404
72                     106473
33                     135139
23                     342346
95               ict II406890
Name: beis_school_id, dtype: object

In [98]:
root_df["numeric_beis"] = isnumeric_filter

In [100]:
root_df[root_df["numeric_beis"]]

ValueError: Cannot mask with non-boolean array containing NA / NaN values